In [ ]:
install.packages("dismo")
install.packages("ggplot2")
install.packages("dplyr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘raster’, ‘sp’, ‘Rcpp’, ‘terra’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Load packages
library(dismo) # species distribution model (loads raster, sp)
library(dplyr) # data manipulation package
library(ggplot2) # graphing package

Loading required package: raster

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, were retired in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.


Attaching package: ‘dplyr’


The following objects are masked from ‘package:raster’:

    intersect, select, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
# Load current environmental data
lanternfly_data = read.csv("./test_lanternfly_data.csv")
env_data_current = stack("./env_current.grd")
env_data_forecast = stack("./env_forecast.grd")

Warning message in file(file, "rt"):
“cannot open file './test_lanternfly_data.csv': No such file or directory”


ERROR: ignored

In [ ]:
# tmin : min temperatures
# precip : precipitations
plot(env_data_current$tmin) # outputs a raster map on min temperatures
plot(env_data_current$precip) # outputs a raster map on precipitation

In [ ]:
# BACKGROUND DATA : Randomly generated absence data
# raster layer to sample from
mask <- raster("./env_current.grd")
# set seed to assure that the dataset will always have the same random sample.
set.seed(1963)

# set coordinate limits to USA
e <- extent(-140, -50, 25, 60)

# select 17731 random points
bg <- randomPoints(mask, 17731, ext=e)
bg = as.data.frame(bg)

# rename the columns and add the two that aren't present
colnames(bg)[colnames(bg) == "x"] <- "lon"
colnames(bg)[colnames(bg) == "y"] <- "lat"
bg['present'] <- 0
bg['ObsDate'] <- 0

# put columns in right order
bg <- bg[, c("ObsDate", "lat", "lon", "present")]

# bind random absence data with presence data
lanternfly_data = rbind(lanternfly_data, bg)

In [ ]:
# select just the location columns data:
lanternfly_locations = select(lanternfly_data, lon, lat)
# use extract function with current environment data and locations where we want to extract our data
lanternfly_env = extract(env_data_current, lanternfly_locations)
# bind together as columns our current warb data with our current environmental conditions data
lanternfly_data = cbind(lanternfly_data, lanternfly_env)

# Plot our current warb data, with temperature (x) and precipitation (y) and presence of warbs (color [darker - absent, lighter - present])
# species likes higher temperature and precipitation
ggplot(lanternfly_data, mapping = aes(x = tmin,
                                       y = precip,
                                       color = present)) + geom_point()

In [ ]:
# 5. Build a species distribution model: multivariate logistic regression model
# takes in the model to be used (present var as response) related to tmin and precip variables
# family model ~ binomial family with link logistic to model the data
logistic_regr_model = glm(present ~ tmin + precip,
                          family = binomial(link = "logit"),
                          data = lanternfly_data)

# Summary of logistic regression model
summary(logistic_regr_model)

# split our data between presences and absences
presence_data = filter(lanternfly_data, present == 1)
absence_data = filter(lanternfly_data, present == 0)

# use evaluate function to calculate model performance (ROC curve evaluation: true positives, false positives)
evaluation = evaluate(presence_data,
                      absence_data,
                      logistic_regr_model)
plot(evaluation, 'ROC')

In [ ]:
# Spacial prediction environmental conditions to make predictions on + regression model + type of display (probs)
predictions = predict(env_data_current,
                      logistic_regr_model,
                      type = "response")

# 6. Plot prediction for the USA
plot(predictions, ext = extent(-140, -50, 25, 60))
points(presence_data[c("lon", "lat")], pch = "+", cex = 0.5)

# Predict locations likely to exist - plot predicted species range
plot(predictions > 0.5, ext = extent(-140, -50, 25, 60))

In [ ]:
# Select the appropriate threshold value  and plot the prediction of locations where warbs are likely to exist
tr = threshold(evaluation, stat = 'prevalence')
plot(predictions > tr, ext = extent(-140, -50, 25, 60))
points(presence_data[c("lon", "lat")], pch = "+", cex = 0.5)

In [ ]:
# 7. Make forecasts using future environmental data using a logistic regression model
forecasts = predict(env_data_forecast,
                    logistic_regr_model,
                    type = "response")

# plot forecast model in the USA with the calculated threshold
plot(forecasts, ext = extent(-140, -50, 25, 60))
plot(forecasts > tr, ext = extent(-140, -50, 25, 60))

# Observe predicted changes in probabilities over the next 50 years
plot(forecasts - predictions, ext = extent(-140, -50, 25, 60))